In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import os
import pickle
from sklearn.metrics import confusion_matrix
import requests, bs4
from sklearn.metrics import roc_curve, auc

In [35]:
db = pd.read_csv('2017_2018stats.csv')

In [36]:
db19 = pd.read_csv('2019stats.csv')

### Training Naive Bayes classifier on 2016-2018 data ###
Model Variations: 3 game rolling averages, 9 game rolling averages

In [37]:
feature3Cols = ['Location','Rolling3EFG','Rolling3TOV','Rolling3ORB','Rolling3FTR','Rolling3Pace','Opp R3Pace','Opp R3EFG','Opp R3TOV','Opp R3ORB','Opp R3FTR', 'Rolling3DEff', 'Opp R3 Deff']
feature9Cols = ['Location','Rolling9EFG','Rolling9TOV','Rolling9ORB','Rolling9FTR','Rolling9Pace','Opp R9Pace','Opp R9EFG','Opp R9TOV','Opp R9ORB','Opp R9FTR', 'Rolling9DEff', 'Opp R9 Deff']

target = ['Result']
X3 = db.dropna()[feature3Cols]
X9 = db.dropna()[feature9Cols]
y = np.array(db.dropna()[target]).flatten()

x9_train, x9_test, y9_train, y9_test = train_test_split(X9, y, test_size=0.3, random_state=1)
x3_train, x3_test, y3_train, y3_test = train_test_split(X3, y, test_size=0.3, random_state=1)

In [29]:
GNB9 = GaussianNB().fit(x9_train, y9_train)
pred = GNB9.predict(x9_test)
print('All picks (R9):',GNB9.score(x9_test, y9_test))
print('All picks (R9) matrix:\n',confusion_matrix(y9_test, pred))
print(' ')

GNB3 = GaussianNB().fit(x3_train, y3_train)
pred = GNB3.predict(x3_test)
print('All picks (R3):',GNB3.score(x3_test, y3_test))
print('All picks (R3) matrix:\n',confusion_matrix(y3_test, pred))
print(' ')


All picks (R9): 0.6514459665144596
All picks (R9) matrix:
 [[423 227]
 [231 433]]
 
All picks (R3): 0.6423135464231354
All picks (R3) matrix:
 [[420 230]
 [240 424]]
 


In [30]:
dogs2019 = db19[db19['Line'] > 0].dropna().reset_index(drop = True)
Ydogs2019 = db19[db19['Line'] > 0].dropna()['Result'].reset_index(drop=True)

In [10]:
Ydogs2019.head()

0    0
1    0
2    0
3    1
4    0
Name: Result, dtype: int64

In [31]:
print("Using dog data[rolling 9 averages]: \n")

print("Using 9 game standard predictor: ")
pred1 = GNB9.predict(dogs2019[feature9Cols])
print('Dog picks:',GNB9.score(dogs2019[feature9Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred1))
print('')

print("Using 3 game standard predictor: ")
pred = GNB3.predict(dogs2019[feature9Cols])
print('Dog picks:',GNB3.score(dogs2019[feature9Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')


Using dog data[rolling 9 averages]: 

Using 9 game standard predictor: 
Dog picks: 0.6234855545200373
Dog picks matrix:
 [[548 168]
 [236 121]]

Using 3 game standard predictor: 
Dog picks: 0.6328052190121156
Dog picks matrix:
 [[563 153]
 [241 116]]



In [32]:
odds = []
for i, n in enumerate(pred1):
    if n == 1:
        odds.append(dogs2019['Odds'][i])
import statistics
statistics.mean(odds)

155

In [33]:
print("Using rolling 3 averages: \n")

print("Using 9 game standard predictor: ")
pred = GNB9.predict(dogs2019[feature3Cols])
print('Dog picks:',GNB9.score(dogs2019[feature3Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using 3 game standard predictor: ")
pred = GNB3.predict(dogs2019[feature3Cols])
print('Dog picks:',GNB3.score(dogs2019[feature3Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')


Using rolling 3 averages: 

Using 9 game standard predictor: 
Dog picks: 0.608574091332712
Dog picks matrix:
 [[488 228]
 [192 165]]

Using 3 game standard predictor: 
Dog picks: 0.6244175209692451
Dog picks matrix:
 [[525 191]
 [212 145]]



3 game sampler is not great

In [21]:
odds = []
for i, n in enumerate(pred1):
    if n == 1:
        odds.append(dogs2019['Odds'][i])
statistics.mean(odds)

155

In [23]:
pred_prob = GNB9.predict_proba(dogs2019[feature9Cols])

Average odds for all dog predictions was +166

Betting 100 on each 'yes' prediction is 28600 spent on total bets, with 166 returned on each win.

#### Winnings: 20584    Losses: 16200    Profits: 4384    15% ROI

In [55]:
pkl_filename = "GNB9model.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(GNB9, file)

In [12]:
db = pd.read_csv('2017_2018stats.csv')

In [13]:
feature3Cols = ['Location','Rolling3EFG','Rolling3TOV','Rolling3ORB','Rolling3FTR','Rolling3Pace','Opp R3Pace','Opp R3EFG','Opp R3TOV','Opp R3ORB','Opp R3FTR']
feature9Cols = ['Location','Rolling9EFG','Rolling9TOV','Rolling9ORB','Rolling9FTR','Rolling9Pace','Opp R9Pace','Opp R9EFG','Opp R9TOV','Opp R9ORB','Opp R9FTR']

target = ['Result']
X3 = db.dropna()[feature3Cols]
X9 = db.dropna()[feature9Cols]
y = np.array(db.dropna()[target]).flatten()

x9_train, x9_test, y9_train, y9_test = train_test_split(X9, y, test_size=0.3, random_state=1)
x3_train, x3_test, y3_train, y3_test = train_test_split(X3, y, test_size=0.3, random_state=2)


### Training models

In [ ]:
pkl_filename = "GNB9model.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(GNB9, file)